In [1]:
from msibi import MSIBI, State, Pair, Bond, Angle
from cmeutils.structure import angle_distribution, bond_distribution, gsd_rdf
import gsd
import gsd.hoomd
import time
import matplotlib.pyplot as plt
import math
import numpy as np
import signac
import os
import shutil

from math import factorial


colors = [
    "#f6ab17", "#fb751a", "#f8571a", "#ee3a1a",
    "#d6241b", "#b41b1f", "#8b202b", "#642c41",
    "#483e5f", "#37517f", "#2280b6", "#0cbaf7",
    
]

colors2 = [
    "#f6ab17", "#fb751a", "#f8571a", "#ee3a1a",
    "#d6241b", "#b41b1f", "#8b202b", "#642c41",
    "#483e5f", "#37517f", "#2280b6", "#0cbaf7",
    "#f9c75e", "#fc9c5c", "#fa885c", "#f3755f",
    "#d6736d", "#e44e52", "#d64957", "#b65a7c",
    "#8375a4", "#6886bd", "#58ade0", "#56d1fa"
    
]

def savitzky_golay(y, window_size, order, deriv=0, rate=1):
    """Smoothing function used for potentials and distributons
    
    Parameters
    ----------
    y:
    window_size:
    order:
    deriv:
    rate:

    Returns
    -------

    """
    if not (isinstance(window_size, int) and isinstance(order, int)):
        raise ValueError("window_size and order must be of type int")
    if window_size % 2 != 1 or window_size < 1:
        raise TypeError("window_size must be a positive odd number")
    if window_size < order + 2:
        raise TypeError("window_size is too small for the polynomials order")

    order_range = range(order + 1)
    half_window = (window_size - 1) // 2
    b = np.mat(
        [
            [k ** i for i in order_range]
            for k in range(-half_window, half_window + 1)
        ]
    )
    m = np.linalg.pinv(b).A[deriv] * rate ** deriv * factorial(deriv)
    firstvals = y[0] - np.abs(y[1 : half_window + 1][::-1] - y[0])
    lastvals = y[-1] + np.abs(y[-half_window - 1 : -1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve(m[::-1], y, mode="valid")

# Find the non Para/Meta sensitive potentials first

- The idea is that these distributions don't change (significantly) with changing para/meta ratios.
- We still have to pick a ratio to use, I think it makes the most sense to use all Para since any slight
    differences from introducing Meta are probably actually due to the EKK angle, which will be optimized 
    specifically to the P/M ratio of the system.

In [2]:
%%bash

for dir in "states" "rdfs" "potentials"
do
    if [ -d $dir ]
    then
        rm -r $dir
    fi
done

In [1]:
# Call the signac project that contains the single-chain, low density target UA simulation
project = signac.get_project("../learning-runs/single-two-chains")
project.detect_schema()

# Find the signac workspace that contains the PEKK simulation with a para weight of 1.0 (all para)
all_para = [job for job in project.find_jobs(
    {"polymer_lengths": [16],
     "n_compounds": [1],
     "kT_quench": 6.5,
     "para_weight": 1.0}
)][0]

single_chain_runs = [job for job in project if job.sp.n_compounds == [1]]

all_para = [job for job in project.find_jobs(
    {"polymer_lengths": [16],
     "n_compounds": [1],
     "kT_quench": 6.5,
     "para_weight": 1.0}
)][0]

NameError: name 'signac' is not defined

In [ ]:
all_para

# Performing IBI on bond stretching potentials

In [4]:
## Create MSIBI manager class.  Sets simulation specific parameters
n_steps = 1e6
opt = MSIBI(
    integrator="hoomd.md.integrate.nvt",
    nlist="hoomd.md.nlist.cell",
    integrator_kwargs={"tau": 0.1},
    dt=0.0003,
    gsd_period=int(n_steps/500),
    n_steps=n_steps,
)

## Create State object, and add it to the opt.states attribute
## Only using a single state to optimize bonded potentials
opt.add_state(
    State(
        name="A",
        kT=6.5,
        traj_file=single_chain_runs[0].fn("components.gsd"),
        alpha=1.0,
        max_frames=100,
    )
)

## Create Pair objects, and add them to the opt.pairs attribute
## For optimizing the bond-stretching potential, pair potentials are "turned off" (LJ potential with epsilon=0)
pair0 = Pair(type1="E", type2="E")
pair1 = Pair(type1="K", type2="K")
pair2 = Pair(type1="E", type2="K")
for pair in [pair0, pair1, pair2]:
    pair.set_lj(epsilon=0, sigma=1, r_cut=0)
    opt.add_pair(pair)

    
## Create Bond objects, and add tem to the opt.bonds attribute
## Using a simple harmonic potential for the initial "guess" potential
bond0 = Bond(type1="E", type2="K", head_correction_form="linear")
bond1 = Bond(type1="K", type2="K", head_correction_form="linear")
bond0.set_quadratic(k2=50, l0=1.51, k3=0, k4=0, l_min=0.0, l_max=4.0, n_points=100)
bond1.set_quadratic(k2=50, l0=1.55, k3=0, k4=0, l_min=0.0, l_max=4.0, n_points=100)
opt.add_bond(bond0)
opt.add_bond(bond1)

files_path = "/home/chris/cme/projects/pekk-msibi/model-2/cg-potentials/angles/boltzmann_inverse/6.5kT/"

#angle0 = Angle(type1="E", type2="K", type3="K", head_correction_form="linear")
#angle1 = Angle(type1="K", type2="E", type3="K", head_correction_form="linear")
#angle0.set_from_file(os.path.join(files_path, "E-K-K_angle_1.0.txt"))
#angle1.set_from_file(os.path.join(files_path, "K-E-K_angle_1.0.txt"))
3opt.add_angle(angle0)
#opt.add_angle(angle1)

## Run the optimization
opt.optimize_bonds(n_iterations=12)

---Bond Optimization: 1 of 12---
Running state A on GPU
Launched HOOMD in /home/chris/cme/projects/pekk-msibi/model-2/notebooks/states/A_6.5



KeyboardInterrupt



# Performing IBI on bond angle potentials:

In [ ]:

for idx, job in enumerate(single_chain_runs[0]):
    n_steps = 1e6
    weight = job.sp.para_weight
    print(f"Run {idx} for para weight {weight}")
    opt = MSIBI(
        nlist="hoomd.md.nlist.tree",
        integrator="hoomd.md.integrate.nvt",
        integrator_kwargs={"tau": 0.01},
        dt=0.0001,
        gsd_period=int(n_steps/500),
        n_steps=n_steps,
        max_frames=250
    )
    ## Create State object, and add it to the opt.states attribute
    ## Only using a single state to optimize bonded potentials
    opt.add_state(
        State(
            name="A",
            kT=job.sp.kT_quench,
            traj_file=job.fn("components.gsd"), alpha=1.0
        )
    )

    ## Create Pair objects, and add them to the opt.pairs attribute
    ## For optimizing the bond-stretching potential, pair potentials are "turned off" (LJ potential with epsilon=0)
    pair0 = Pair(type1="E", type2="E")
    pair1 = Pair(type1="K", type2="K")
    pair2 = Pair(type1="E", type2="K")
    for pair in [pair0, pair1, pair2]:
        pair.set_lj(epsilon=0, sigma=1, r_cut=0)
        opt.add_pair(pair)

    ## Create Bond objects, and add them to the opt.bonds
    ## Setting bond potentials from file (IBI Bonds run)
    bond0 = Bond(type1="E", type2="K", head_correction_form="linear")
    bond1 = Bond(type1="K", type2="K", head_correction_form="linear")
    bond0.set_from_file(
        file_path=os.path.join(os.getcwd(), "E-K_smoothed.txt")
    )
    bond1.set_from_file(
        file_path=os.path.join(os.getcwd(), "K-K_smoothed.txt")
    )
    opt.add_bond(bond0)
    opt.add_bond(bond1)
    
    # Create Angle objects, and add them to opt.angles
    # Since we are optimizing angles, set quadratic pot with a guess
    angle0 = Angle(type1="E", type2="K", type3="K", head_correction_form="linear")
    angle1 = Angle(type1="K", type2="E", type3="K", head_correction_form="linear")
    angle0.set_quadratic(theta0=2.5, k4=0, k3=0, k2=100)
    angle1.set_quadratic(theta0=2.5, k4=0, k3=0, k2=100)
    opt.add_angle(angle0)
    opt.add_angle(angle1)

    ## Run the optimization
    opt.optimize_angles(n_iterations=12)

    # Set up P/M Dirs and move results
    os.mkdir(os.path.join(os.getcwd(), f"ekk_{weight}"))
    for _dir in ["potentials", "states"]:
        shutil.move(
            os.path.join(os.getcwd(), _dir),
            os.path.join(os.getcwd(), f"ekk_{weight}")
        )
    #os.rename("E-K-K_smoothed.txt", f"E-K-K-{weight}.txt")
    #os.rename("K-E-K_smoothed.txt", f"K-E-K-{weight}.txt")

$P(l) \propto \exp\left[-\dfrac{U(l)}{k_BT}\right]$

$P(\alpha) \propto \exp\left[-\dfrac{U(\alpha)}{k_BT}\right]$

$P(\theta) \propto \exp\left[-\dfrac{U(\theta)}{k_BT}\right]$

$g(r_{i,j}) \propto \exp\left[-\dfrac{U(r_{i,j})}{k_BT}\right]$